In [217]:
import pandas as pd
import numpy as np


In [218]:
amazon_price = pd.read_csv("AMZN.csv")
amazon_eps = pd.read_csv("earnings_latest.csv").groupby("symbol").get_group("AMZN")
amazon_eps["date"] = pd.to_datetime(amazon_eps["date"])
amazon_price["Date"] = pd.to_datetime(amazon_price["Date"])
start_date = "2009-07-23"
end_date = "2021-04-29"
amazon_price = amazon_price[(amazon_price["Date"] >= start_date) & (amazon_price["Date"] <= end_date)]
amazon_price["election_season"] = (
    ((amazon_price["Date"].dt.year % 4) == 0) & 
    (amazon_price["Date"].dt.month > 7) & 
    (amazon_price["Date"].dt.month < 12)
).astype(int)

In [219]:
amazon_eps["date"] = pd.to_datetime(amazon_eps["date"])
amazon_price["Date"] = pd.to_datetime(amazon_price["Date"])

In [220]:
eps_values = []
eps_est_values = []

for date in amazon_price["Date"]:
    relevant_eps = amazon_eps[amazon_eps["date"] <= date]
    relevant_eps_est = amazon_eps[amazon_eps["date"] <= date]
    if not relevant_eps.empty:
        eps_values.append(relevant_eps.iloc[-1]["eps"])
        eps_est_values.append(relevant_eps_est.iloc[-1]["eps_est"])
    else:
        eps_values.append(None)
        eps_est_values.append(None)

amazon_price["eps"] = eps_values
amazon_price["eps_est"] = eps_est_values

In [221]:
amazon_price["earnings_beat"] = amazon_price["eps"] > amazon_price["eps_est"]
amazon_price["earnings_beat"] = amazon_price["earnings_beat"].astype(int)
amazon_eps

,symbol,date,qtr,eps_est,eps,release_time
8735,AMZN,2009-07-23,06/2009,NaN,NaN,post
8736,AMZN,2009-10-22,09/2009,NaN,NaN,post
8737,AMZN,2010-01-28,12/2009,NaN,NaN,post
8738,AMZN,2010-04-22,03/2010,NaN,NaN,post
8739,AMZN,2010-07-22,06/2010,NaN,NaN,post
8740,AMZN,2010-10-21,09/2010,NaN,NaN,post
8741,AMZN,2011-01-27,12/2010,NaN,NaN,post
8742,AMZN,2011-04-26,03/2011,NaN,NaN,NaN
8743,AMZN,2011-07-26,06/2011,NaN,NaN,post
8744,AMZN,2011-10-25,09/2011,NaN,NaN,post


In [222]:
release_times = []
for date in amazon_price["Date"]:
    relevant_eps = amazon_eps[amazon_eps["date"] <= date]
    if not relevant_eps.empty:
        latest_release = relevant_eps.iloc[-1]
        if latest_release["release_time"] == 0: 
            release_times.append((latest_release["date"] - pd.Timedelta(days=1)).date())
        else:  
            release_times.append(latest_release["date"].date())
    else:
        release_times.append(None)
release_times_df = pd.DataFrame({"Date": amazon_price["Date"], "release_time_date": release_times})
amazon_price = amazon_price.merge(release_times_df, on="Date", how="left")

amazon_price["release_time"] = (amazon_price["Date"].dt.date == amazon_price["release_time_date"]).astype(int)

amazon_price.drop(columns=["release_time_date"], inplace=True)


In [223]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
amazon_price


,Date,Open,High,Low,Close,Adj Close,Volume,election_season,eps,eps_est,earnings_beat,release_time
0,2009-07-23,4.492000,4.720000,4.479500,4.693500,4.693500,383242000,0,NaN,NaN,0,1
1,2009-07-24,4.380000,4.445000,4.275000,4.324500,4.324500,384092000,0,NaN,NaN,0,0
2,2009-07-27,4.312000,4.324500,4.178000,4.212000,4.212000,225586000,0,NaN,NaN,0,0
3,2009-07-28,4.192000,4.282000,4.130000,4.249000,4.249000,175498000,0,NaN,NaN,0,0
4,2009-07-29,4.223500,4.275000,4.176000,4.217000,4.217000,130058000,0,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2958,2021-04-23,165.955002,168.750000,165.425003,167.044006,167.044006,63856000,0,14.09,7.231,1,0
2959,2021-04-26,167.399994,171.422501,166.546997,170.449997,170.449997,97614000,0,14.09,7.231,1,0
2960,2021-04-27,172.173492,173.000000,169.900497,170.871506,170.871506,76542000,0,14.09,7.231,1,0
2961,2021-04-28,171.740005,174.494003,171.250000,172.925003,172.925003,92638000,0,14.09,7.231,1,0


In [224]:
model = RandomForestClassifier(n_estimators=100, min_samples_split=100, random_state=1)
train = amazon_price.iloc[:-100]
test = amazon_price.iloc[-100:]
predictors = ["Open", "High", "Low", "Close",  "Adj Close", "Volume", "election_season", ]